# 03) 멀티오믹스 Cis-correlation 분석

```
NB02 (생존분석) → NB03 (cis-correlation) → results/nb03_* → NB04
```

**핵심 개념**: 같은 유전자의 mRNA 발현과 프로모터 메틸화 간 상관분석 (cis = 같은 유전자)

## 배경: Cis-correlation이란?

| 패턴 | 메틸화 위치 | 발현 관계 | 생물학적 의미 |
|------|-----------|----------|-------------|
| **프로모터 과메틸화** | TSS200, TSS1500, 5'UTR | **음의 상관** (ρ < 0) | 종양억제유전자 침묵 |
| **유전자체 메틸화** | Gene Body, 3'UTR | **양의 상관** (ρ > 0) | 활발한 전사 표지 |

### Trans vs Cis
| 방법 | 비교 대상 | 검정 수 | 문제점 |
|------|----------|---------|--------|
| Trans (기존) | 모든 유전자 × 모든 프로브 | ~수십만 개 | 다중검정 부담, 해석 어려움 |
| **Cis (본 분석)** | **같은 유전자**의 발현 vs 메틸화 | ~1,000개 | 검정 수 감소 → 통계적 파워 ↑ |

## 라이브러리 및 환경 설정

In [ ]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from scipy.stats import spearmanr, mannwhitneyu, kruskal
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

try:
    from statsmodels.stats.multitest import multipletests
except ImportError:
    def multipletests(pvals, alpha=0.05, method='fdr_bh'):
        n = len(pvals)
        idx = np.argsort(pvals)
        sorted_p = np.array(pvals)[idx]
        corrected = np.empty(n)
        corrected[idx[-1]] = sorted_p[-1]
        for i in range(n - 2, -1, -1):
            corrected[idx[i]] = min(corrected[idx[i + 1]], sorted_p[i] * n / (i + 1))
        return corrected < alpha, corrected, None, None

def sig_stars(p, ns=''):
    if p < 0.001: return '***'
    if p < 0.01:  return '**'
    if p < 0.05:  return '*'
    return ns

def make_annot(corr_df, pval_df):
    annot = pd.DataFrame('', index=corr_df.index, columns=corr_df.columns)
    for r in corr_df.index:
        for c in corr_df.columns:
            v, p = corr_df.loc[r, c], pval_df.loc[r, c]
            if pd.notna(v):
                annot.loc[r, c] = f'{v:.2f}{sig_stars(p)}'
    return annot

import plot_config
plot_config.setup()
np.random.seed(42)

## 1. 데이터 로드

| 파일 | 내용 | 형태 |
|------|------|------|
| `input/mrna_matched.txt` | mRNA 발현 | gene × sample |
| `input/methylation_gene_level.txt` | **유전자별 메틸화** (TSS 프로브 우선 집계) | gene × sample |
| `input/probe_gene_mapping.tsv` | 프로브→유전자 매핑 | probe, gene, location |
| `input/clinical_matched.tsv` | 임상 정보 | patient × variable |

In [ ]:
# 파일 경로
MRNA = "input/mrna_matched.txt"
CLIN = "input/clinical_matched.tsv"
METH_GENE = "input/methylation_gene_level.txt"
PROBE_MAP = "input/probe_gene_mapping.tsv"
SAMPLE_COL = "Sample ID"
RESULT_DIR = "results"

# 데이터 로드
mrna = pd.read_csv(MRNA, sep="\t", index_col=0)
clinical = pd.read_csv(CLIN, sep="\t")
meth_gene = pd.read_csv(METH_GENE, sep="\t", index_col=0)
probe_info = pd.read_csv(PROBE_MAP, sep="\t")

# 중복 유전자 제거 (분산이 가장 큰 행 유지)
if mrna.index.duplicated().any():
    dup_count = mrna.index.duplicated(keep=False).sum()
    print(f"[주의] mRNA에 중복 행 {dup_count}개 → 유전자당 분산 최대 행만 유지")
    mrna['_var'] = mrna.var(axis=1)
    mrna = mrna.sort_values('_var', ascending=False)
    mrna = mrna[~mrna.index.duplicated(keep='first')]
    mrna = mrna.drop(columns='_var')

# 공통 샘플 정렬
samples = sorted(set(mrna.columns) & set(meth_gene.columns) & set(clinical[SAMPLE_COL]))
mrna = mrna[samples]
meth_gene = meth_gene[samples]
clinical = clinical[clinical[SAMPLE_COL].isin(samples)].reset_index(drop=True)

# 공통 유전자 매칭 (cis-correlation 대상)
common_genes = sorted(set(mrna.index) & set(meth_gene.index))
mrna_cis = mrna.loc[common_genes, samples]
meth_cis = meth_gene.loc[common_genes, samples]

# 유전자별 메틸화 위치 (프로모터/Body/Other)
gene_loc = probe_info.groupby('gene')['loc_category'].first()
gene_locations = gene_loc.reindex(common_genes).fillna('Other')

print(f"=== 데이터 요약 ===")
print(f"mRNA: {mrna.shape[0]:,}개 유전자 × {mrna.shape[1]}명")
print(f"Gene-level methylation: {meth_gene.shape[0]:,}개 유전자")
print(f"공통 유전자 (cis 대상): {len(common_genes):,}개")
print(f"  Promoter: {(gene_locations == 'Promoter').sum()}"
      f" | Body: {(gene_locations == 'Body').sum()}"
      f" | Other: {(gene_locations == 'Other').sum()}")

In [ ]:
# 프로브-유전자 매핑 통계 시각화
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# 1) 위치 카테고리 분포
loc_counts = probe_info['loc_category'].value_counts()
axes[0].bar(loc_counts.index, loc_counts.values,
            color=['#e74c3c', '#2ecc71', '#95a5a6'], alpha=0.8)
axes[0].set_title('프로브 위치 분포')
axes[0].set_ylabel('프로브 수')

# 2) 유전자당 프로브 수
probes_per_gene = probe_info.groupby('gene').size()
axes[1].hist(probes_per_gene.clip(upper=20), bins=20,
             color='steelblue', alpha=0.7, edgecolor='white')
axes[1].set_xlabel('유전자당 프로브 수')
axes[1].set_ylabel('유전자 수')
axes[1].set_title(f'유전자당 프로브 분포 (중앙값: {probes_per_gene.median():.0f}개)')

# 3) 공통 유전자 현황
n_mrna_only = len(set(mrna.index) - set(meth_gene.index))
n_meth_only = len(set(meth_gene.index) - set(mrna.index))
n_common = len(common_genes)
labels = [f'mRNA only\n{n_mrna_only}', f'공통\n{n_common}', f'Meth only\n{n_meth_only}']
axes[2].bar(labels, [n_mrna_only, n_common, n_meth_only],
            color=['steelblue', '#2ecc71', 'coral'], alpha=0.8)
axes[2].set_title('mRNA ∩ Methylation 유전자')
axes[2].set_ylabel('유전자 수')

plt.tight_layout()
plt.show()

### Vibe Coding 미션 D1: 프로브-유전자 매핑 탐색
**미션**: 프로브-유전자 매핑 데이터를 더 깊이 탐색해보세요!

**도전 과제**:
1. 프로브가 가장 많은 Top 10 유전자를 막대그래프로 시각화하기
2. 프로모터 프로브만 가진 유전자 vs Body 프로브만 가진 유전자 수 비교하기
3. (도전) 특정 유전자(예: TP53, BRCA1)의 프로브 위치 분포 확인하기

In [ ]:
# 미션 D1 코드를 여기에 작성하세요!

## 2. 단일 유전자 Cis-correlation 예시

**Cis-correlation**: 같은 유전자의 mRNA 발현 ↔ 프로모터 메틸화 간 Spearman 상관

In [ ]:
# 첫 번째 공통 유전자로 시연
gene_ex = common_genes[0]
expr_vals = mrna_cis.loc[gene_ex].values.astype(float)
meth_vals = meth_cis.loc[gene_ex].values.astype(float)
valid = ~(np.isnan(expr_vals) | np.isnan(meth_vals))

rho, pval = spearmanr(expr_vals[valid], meth_vals[valid])

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(expr_vals[valid], meth_vals[valid], alpha=0.6, s=50,
           color='steelblue', edgecolors='white')

# 추세선
z = np.polyfit(expr_vals[valid], meth_vals[valid], 1)
x_line = np.linspace(expr_vals[valid].min(), expr_vals[valid].max(), 100)
ax.plot(x_line, np.poly1d(z)(x_line), 'r--', alpha=0.7, lw=2)

loc_label = gene_locations[gene_ex]
ax.set_xlabel(f"{gene_ex} mRNA 발현량", fontsize=12)
ax.set_ylabel(f"{gene_ex} 메틸화 (β값)", fontsize=12)
ax.set_title(f"Cis-correlation: {gene_ex} ({loc_label})\n"
             f"Spearman ρ = {rho:.3f}, p = {pval:.2e}", fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"유전자: {gene_ex} | 위치: {loc_label}")
print(f"Spearman ρ = {rho:.3f}, p = {pval:.2e}")
if rho < -0.3:
    print("→ 음의 상관: 메틸화↑ → 발현↓ (프로모터 침묵)")
elif rho > 0.3:
    print("→ 양의 상관: 메틸화↑ → 발현↑ (유전자체 활성)")
else:
    print("→ 약한 상관")

## 3. 전체 유전자 Cis-correlation + FDR 보정

모든 공통 유전자에 대해 **Spearman cis-correlation** 계산 후 **Benjamini-Hochberg FDR** 보정

In [ ]:
# 전체 cis-correlation 계산
print(f"총 {len(common_genes):,}개 유전자의 cis-correlation 계산 중...")

cis_results = []
for gene in common_genes:
    expr = mrna_cis.loc[gene].values.astype(float)
    meth = meth_cis.loc[gene].values.astype(float)
    valid = ~(np.isnan(expr) | np.isnan(meth))

    if valid.sum() > 10:
        rho, pval = spearmanr(expr[valid], meth[valid])
    else:
        rho, pval = np.nan, np.nan

    cis_results.append({
        'gene': gene, 'rho': rho, 'p_value': pval,
        'location': gene_locations[gene], 'n_valid': int(valid.sum())
    })

cis_df = pd.DataFrame(cis_results).dropna(subset=['rho'])

# FDR 보정
reject, fdr_vals, _, _ = multipletests(cis_df['p_value'].values, alpha=0.05, method='fdr_bh')
cis_df['fdr'] = fdr_vals
cis_df['significant'] = reject
cis_df = cis_df.sort_values('p_value')

# 결과 요약
n_total = len(cis_df)
n_sig = cis_df['significant'].sum()
n_neg = ((cis_df['rho'] < 0) & cis_df['significant']).sum()
n_pos = ((cis_df['rho'] > 0) & cis_df['significant']).sum()

print(f"\n=== Cis-correlation 결과 ===")
print(f"총 검정: {n_total:,}개 유전자")
print(f"유의 (FDR < 0.05): {n_sig:,}개 ({n_sig/n_total*100:.1f}%)")
print(f"  음의 상관 (프로모터 침묵): {n_neg:,}개")
print(f"  양의 상관 (유전자체 활성): {n_pos:,}개")
print(f"ρ 범위: [{cis_df['rho'].min():.3f}, {cis_df['rho'].max():.3f}]")
print(f"ρ 중앙값: {cis_df['rho'].median():.3f}")

In [ ]:
# Cis-correlation 분포 + Volcano Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# (1) 분포 히스토그램
ax = axes[0]
sns.histplot(cis_df['rho'], bins=40, kde=True, color='skyblue', alpha=0.7, ax=ax)
ax.axvline(0, ls='-', c='gray', alpha=0.5)
ax.axvline(-0.3, ls='--', c='red', lw=2, alpha=0.7, label='ρ = ±0.3')
ax.axvline(0.3, ls='--', c='green', lw=2, alpha=0.7)
ax.set_xlabel("Spearman ρ (cis-correlation)", fontsize=12)
ax.set_ylabel("유전자 수", fontsize=12)
ax.set_title(f"Cis-correlation 분포 (n={len(cis_df):,}, "
             f"중앙값={cis_df['rho'].median():.3f})", fontsize=13)
ax.legend()

# (2) Volcano plot
ax = axes[1]
neg_log_fdr = -np.log10(np.clip(cis_df['fdr'].values, 1e-300, 1))
sig = cis_df['significant'].values
rho_vals = cis_df['rho'].values

ax.scatter(rho_vals[~sig], neg_log_fdr[~sig],
           alpha=0.4, s=20, c='grey', label=f'비유의 ({(~sig).sum():,})')
ax.scatter(rho_vals[sig & (rho_vals < 0)], neg_log_fdr[sig & (rho_vals < 0)],
           alpha=0.6, s=30, c='blue', label=f'음의 상관 ({(sig & (rho_vals < 0)).sum():,})')
ax.scatter(rho_vals[sig & (rho_vals > 0)], neg_log_fdr[sig & (rho_vals > 0)],
           alpha=0.6, s=30, c='red', label=f'양의 상관 ({(sig & (rho_vals > 0)).sum():,})')

ax.axhline(-np.log10(0.05), ls='--', c='orange', alpha=0.5, label='FDR = 0.05')
ax.axvline(-0.3, ls=':', c='gray', alpha=0.4)
ax.axvline(0.3, ls=':', c='gray', alpha=0.4)

# Top 유전자 라벨
for _, row in cis_df.head(5).iterrows():
    ax.annotate(row['gene'], (row['rho'], -np.log10(max(row['fdr'], 1e-300))),
                fontsize=7, alpha=0.8, ha='center', va='bottom')

ax.set_xlabel("Spearman ρ", fontsize=12)
ax.set_ylabel("-log₁₀(FDR)", fontsize=12)
ax.set_title("Volcano Plot: Cis-correlation", fontsize=13)
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

## 4. 프로모터 vs Body 위치별 비교

프로모터(TSS) 메틸화 → **발현 억제** (음의 상관)
유전자체(Body) 메틸화 → **전사 활성** (양의 상관)

In [ ]:
# 위치별 cis-correlation 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# (1) 박스플롯
ax = axes[0]
loc_order = ['Promoter', 'Body', 'Other']
plot_data = cis_df[cis_df['location'].isin(loc_order)]
sns.boxplot(data=plot_data, x='location', y='rho', order=loc_order,
            palette={'Promoter': '#e74c3c', 'Body': '#2ecc71', 'Other': '#95a5a6'}, ax=ax)
ax.axhline(0, ls='--', c='gray', alpha=0.5)
ax.set_xlabel('메틸화 프로브 위치')
ax.set_ylabel('Spearman ρ')
ax.set_title('위치별 Cis-correlation')

prom = cis_df[cis_df['location'] == 'Promoter']['rho'].dropna()
body = cis_df[cis_df['location'] == 'Body']['rho'].dropna()
if len(prom) > 5 and len(body) > 5:
    u, mw_p = mannwhitneyu(prom, body, alternative='two-sided')
    ax.text(0.5, 0.95, f'Promoter vs Body: p={mw_p:.2e} {sig_stars(mw_p, "ns")}',
            transform=ax.transAxes, ha='center', va='top', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# (2) 유의 비율
ax = axes[1]
sig_by_loc = cis_df.groupby('location').agg(
    total=('gene', 'count'), n_sig=('significant', 'sum'),
    mean_rho=('rho', 'mean')
).reindex(loc_order).dropna()
sig_by_loc['sig_pct'] = sig_by_loc['n_sig'] / sig_by_loc['total'] * 100

bars = ax.bar(sig_by_loc.index, sig_by_loc['sig_pct'],
              color=['#e74c3c', '#2ecc71', '#95a5a6'], alpha=0.8)
for bar, (_, row) in zip(bars, sig_by_loc.iterrows()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{row["n_sig"]:.0f}/{row["total"]:.0f}\n(ρ={row["mean_rho"]:.3f})',
            ha='center', va='bottom', fontsize=10)
ax.set_ylabel('유의한 유전자 비율 (%)')
ax.set_title('위치별 유의 비율 (FDR < 0.05)')

plt.tight_layout()
plt.show()

for loc in loc_order:
    sub = cis_df[cis_df['location'] == loc]
    if len(sub) > 0:
        print(f"  {loc:>10}: n={len(sub):>4}, 평균 ρ={sub['rho'].mean():+.3f}, "
              f"유의={sub['significant'].sum()}개 ({sub['significant'].mean()*100:.1f}%)")

### Vibe Coding 미션 D2: 위치별 상관 심화 분석
**미션**: 프로모터와 Body 메틸화의 차이를 더 깊이 분석해보세요!

**도전 과제**:
1. Promoter 유전자 중 가장 강한 음의 상관 Top 5, Body 유전자 중 가장 강한 양의 상관 Top 5 비교하기
2. 위치별 ρ 분포를 KDE plot으로 겹쳐 시각화하기
3. (도전) 프로모터 메틸화와 Body 메틸화를 모두 가진 유전자에서 두 값의 상관 비교하기

In [ ]:
# 미션 D2 코드를 여기에 작성하세요!

## 5. Top Cis-correlation 유전자

In [ ]:
# Top 양의/음의 cis-correlation
TOP_N = 10

top_neg = cis_df[cis_df['rho'] < 0].head(TOP_N)
top_pos = cis_df[cis_df['rho'] > 0].nlargest(TOP_N, 'rho')

print("=== Top 10 음의 cis-correlation (프로모터 침묵) ===")
for _, r in top_neg.iterrows():
    print(f"  {r['gene']:>12}  ρ={r['rho']:+.3f}  FDR={r['fdr']:.2e}  "
          f"{sig_stars(r['fdr'], 'ns')}  [{r['location']}]")

print("\n=== Top 10 양의 cis-correlation (유전자체 활성) ===")
for _, r in top_pos.iterrows():
    print(f"  {r['gene']:>12}  ρ={r['rho']:+.3f}  FDR={r['fdr']:.2e}  "
          f"{sig_stars(r['fdr'], 'ns')}  [{r['location']}]")

In [ ]:
# Top 6 cis-pair scatter (음의 3 + 양의 3)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle("Top Cis-correlation 유전자", fontsize=16, fontweight='bold')

for idx, (_, row) in enumerate(top_neg.head(3).iterrows()):
    ax = axes[0, idx]
    gene = row['gene']
    e = mrna_cis.loc[gene].values.astype(float)
    m = meth_cis.loc[gene].values.astype(float)
    v = ~(np.isnan(e) | np.isnan(m))
    ax.scatter(e[v], m[v], alpha=0.6, s=40, color='blue', edgecolors='white')
    z = np.polyfit(e[v], m[v], 1)
    xl = np.linspace(e[v].min(), e[v].max(), 50)
    ax.plot(xl, np.poly1d(z)(xl), 'r--', alpha=0.7, lw=2)
    ax.set_xlabel(f"{gene} (mRNA)")
    ax.set_ylabel(f"{gene} (Methyl β)")
    ax.set_title(f"음의 상관: {gene}\nρ={row['rho']:.3f} [{row['location']}]",
                 color='blue', fontsize=11)
    ax.grid(True, alpha=0.3)

for idx, (_, row) in enumerate(top_pos.head(3).iterrows()):
    ax = axes[1, idx]
    gene = row['gene']
    e = mrna_cis.loc[gene].values.astype(float)
    m = meth_cis.loc[gene].values.astype(float)
    v = ~(np.isnan(e) | np.isnan(m))
    ax.scatter(e[v], m[v], alpha=0.6, s=40, color='red', edgecolors='white')
    z = np.polyfit(e[v], m[v], 1)
    xl = np.linspace(e[v].min(), e[v].max(), 50)
    ax.plot(xl, np.poly1d(z)(xl), 'r--', alpha=0.7, lw=2)
    ax.set_xlabel(f"{gene} (mRNA)")
    ax.set_ylabel(f"{gene} (Methyl β)")
    ax.set_title(f"양의 상관: {gene}\nρ={row['rho']:.3f} [{row['location']}]",
                 color='red', fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 아형별 Cis-correlation 비교

같은 유전자라도 암 아형에 따라 메틸화-발현 관계가 다를 수 있습니다.

In [ ]:
# 아형별 cis-correlation 비교
subtypes = clinical.set_index(SAMPLE_COL).loc[samples, 'Subtype']
main_subtypes = [s for s in subtypes.value_counts().index if (subtypes == s).sum() >= 10][:4]

# Top 음의 상관 유전자 1개로 시연
ex_gene = top_neg.iloc[0]['gene']
e_all = mrna_cis.loc[ex_gene]
m_all = meth_cis.loc[ex_gene]

n_panels = 1 + len(main_subtypes)
fig, axes = plt.subplots(1, n_panels, figsize=(5 * n_panels, 4.5))
fig.suptitle(f"{ex_gene}: 아형별 cis-correlation", fontsize=14, fontweight='bold')

colors_sub = ['steelblue', '#e74c3c', '#2ecc71', '#3498db', '#f39c12']

# 전체
ax = axes[0]
v = ~(e_all.isna() | m_all.isna())
rho_all, _ = spearmanr(e_all[v], m_all[v])
ax.scatter(e_all[v], m_all[v], alpha=0.5, s=30, c=colors_sub[0], edgecolors='white')
ax.set_title(f"전체 (n={v.sum()})\nρ = {rho_all:.3f}", fontsize=11)
ax.set_xlabel(f"{ex_gene} (mRNA)")
ax.set_ylabel(f"{ex_gene} (Methyl)")
ax.grid(True, alpha=0.3)

# 아형별
for idx, st in enumerate(main_subtypes):
    ax = axes[idx + 1]
    mask = subtypes == st
    e_sub, m_sub = e_all[mask], m_all[mask]
    v_sub = ~(e_sub.isna() | m_sub.isna())
    if v_sub.sum() > 5:
        rho_st, _ = spearmanr(e_sub[v_sub], m_sub[v_sub])
    else:
        rho_st = np.nan
    ax.scatter(e_sub[v_sub], m_sub[v_sub], alpha=0.6, s=35,
               c=colors_sub[idx + 1], edgecolors='white')
    ax.set_title(f"{st} (n={v_sub.sum()})\nρ = {rho_st:.3f}", fontsize=11)
    ax.set_xlabel(f"{ex_gene} (mRNA)")
    ax.set_ylabel(f"{ex_gene} (Methyl)")
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 아형별 전체 cis-correlation 분포 비교
fig, ax = plt.subplots(figsize=(10, 5))
for st in main_subtypes:
    mask = subtypes == st
    rhos_st = []
    for gene in common_genes:
        e = mrna_cis.loc[gene][mask]
        m = meth_cis.loc[gene][mask]
        v = ~(e.isna() | m.isna())
        if v.sum() > 10:
            r, _ = spearmanr(e[v], m[v])
            rhos_st.append(r)
    sns.kdeplot(rhos_st, label=f'{st} (n={mask.sum()})', ax=ax, fill=True, alpha=0.3)

ax.axvline(0, ls='--', c='gray', alpha=0.5)
ax.set_xlabel("Spearman ρ (cis-correlation)")
ax.set_ylabel("밀도")
ax.set_title("아형별 Cis-correlation 분포", fontsize=14)
ax.legend()
plt.tight_layout()
plt.show()

### Vibe Coding 미션 D3: 아형 특이적 cis-correlation
**미션**: 특정 아형에서만 강한 cis-correlation을 보이는 유전자를 찾아보세요!

**도전 과제**:
1. 각 아형별로 cis-correlation을 계산하여 아형 간 ρ 차이가 큰 유전자 찾기
2. 해당 유전자의 아형별 산점도를 시각화하기
3. (도전) 아형별 유의한 cis-correlation 유전자 수를 비교하는 막대그래프 만들기

In [ ]:
# 미션 D3 코드를 여기에 작성하세요!

## 7. Top 유전자 × 임상변수 연관

In [ ]:
# Top 음의 상관 유전자 × 암 관련 연속형 임상변수
neg_genes = cis_df[cis_df['rho'] < 0].head(10)['gene'].values
clin_indexed = clinical.set_index(SAMPLE_COL).loc[samples]

# 암종 특성과 관련 높은 변수 선택
cont_candidates = [
    'Diagnosis Age',             # 진단 연령
    'Fraction Genome Altered',   # 유전체 불안정성
    'Aneuploidy Score',          # 염색체 불안정성
    'TMB (nonsynonymous)',       # 종양 변이 부담
    'Mutation Count',            # 돌연변이 수
    'Buffa Hypoxia Score',       # 종양 저산소 환경
    'MSIsensor Score',           # 현미부수체 불안정성
    'Tumor Break Load',          # 유전체 구조 변이
]
cont_vars = [v for v in cont_candidates if v in clin_indexed.columns]

if len(cont_vars) > 0 and len(neg_genes) > 0:
    corr_gc = pd.DataFrame(index=neg_genes, columns=cont_vars, dtype=float)
    pval_gc = pd.DataFrame(index=neg_genes, columns=cont_vars, dtype=float)

    for gene in neg_genes:
        expr = mrna_cis.loc[gene]
        for var in cont_vars:
            vals = pd.to_numeric(clin_indexed[var], errors='coerce')
            valid = expr.notna() & vals.notna()
            if valid.sum() > 10:
                r, p = spearmanr(expr[valid], vals[valid])
                corr_gc.loc[gene, var] = r
                pval_gc.loc[gene, var] = p

    corr_gc = corr_gc.astype(float)
    pval_gc = pval_gc.astype(float)
    annot = make_annot(corr_gc, pval_gc)

    fig, ax = plt.subplots(figsize=(max(10, len(cont_vars)*2), max(6, len(neg_genes)*0.6)))
    sns.heatmap(corr_gc, annot=annot.values, fmt='',
                cmap='RdBu_r', center=0, vmin=-0.5, vmax=0.5,
                linewidths=0.5, ax=ax,
                cbar_kws={'label': 'Spearman ρ', 'shrink': 0.8})
    ax.set_title("Top 음의 cis-corr 유전자 × 암 관련 임상변수\n(* p<0.05, ** p<0.01, *** p<0.001)",
                 fontsize=13)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha='right')
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    plt.tight_layout()
    plt.show()

## 8. Cis-correlation 히트맵

유의한 유전자들의 발현 + 메틸화 패턴을 한 눈에 비교

In [ ]:
# 유의한 유전자 중 |ρ|가 큰 순서로 Top 20 선택
sig_genes = cis_df[cis_df['significant']].copy()
sig_genes['abs_rho'] = sig_genes['rho'].abs()
hm_genes = sig_genes.nlargest(20, 'abs_rho')['gene'].values

if len(hm_genes) < 10:
    hm_genes = cis_df.iloc[:20]['gene'].values

# 발현 + 메틸화 z-score
expr_hm = StandardScaler().fit_transform(mrna_cis.loc[hm_genes].T).T
meth_hm = StandardScaler().fit_transform(meth_cis.loc[hm_genes].T).T

# Cis-corr 사이드바
cis_vals = cis_df.set_index('gene').loc[hm_genes, 'rho'].values

fig, axes = plt.subplots(1, 2, figsize=(18, max(8, len(hm_genes)*0.3)),
                          gridspec_kw={'width_ratios': [1, 1]})

# 발현 히트맵
sns.heatmap(pd.DataFrame(expr_hm, index=hm_genes),
            cmap='RdBu_r', center=0, vmin=-3, vmax=3,
            xticklabels=False, yticklabels=True, ax=axes[0],
            cbar_kws={'label': 'z-score', 'shrink': 0.5})
axes[0].set_title('mRNA 발현 (z-score)', fontweight='bold')
axes[0].set_ylabel('')

# 메틸화 히트맵
sns.heatmap(pd.DataFrame(meth_hm, index=hm_genes),
            cmap='RdBu_r', center=0, vmin=-3, vmax=3,
            xticklabels=False, yticklabels=False, ax=axes[1],
            cbar_kws={'label': 'z-score', 'shrink': 0.5})
axes[1].set_title('Methylation (z-score)', fontweight='bold')

plt.suptitle(f'Top {len(hm_genes)} Cis-correlation 유전자: 발현 vs 메틸화',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## NB04를 위한 결과 저장

In [ ]:
import os
os.makedirs(RESULT_DIR, exist_ok=True)

# 1) 공통 유전자 리스트 (NB04에서 사용)
pd.Series(common_genes).to_csv(
    f"{RESULT_DIR}/nb03_selected_genes.txt", index=False, header=False)

# 2) 프로브 리스트 (NB04에서 사용: 공통 유전자에 매핑된 프로브)
mapped_probes = probe_info[probe_info['gene'].isin(common_genes)]['probe'].unique()
meth_matched = pd.read_csv("input/methylation_matched.txt", sep="\t", index_col=0)
available_probes = sorted(set(mapped_probes) & set(meth_matched.index))
if len(available_probes) == 0:
    available_probes = meth_matched.var(axis=1).nlargest(500).index.tolist()
pd.Series(available_probes).to_csv(
    f"{RESULT_DIR}/nb03_selected_probes.txt", index=False, header=False)

print(f"저장 완료: {RESULT_DIR}/")
print(f"  nb03_selected_genes.txt   ({len(common_genes)}개 유전자)")
print(f"  nb03_selected_probes.txt  ({len(available_probes)}개 프로브)")

---
## 분석 요약

| 단계 | 내용 |
|------|------|
| 데이터 준비 | Gene-level methylation (TSS 우선 집계) + mRNA 공통 유전자 매칭 |
| Cis-correlation | 공통 유전자별 mRNA 발현 ↔ 메틸화 Spearman 상관 |
| FDR 보정 | Benjamini-Hochberg (검정 수 = 공통 유전자 수) |
| 위치별 비교 | Promoter (음의 상관) vs Body (양의 상관) 패턴 확인 |
| 아형별 분석 | 아형에 따른 cis-correlation 차이 |
| 임상 연관 | Top 유전자 × 임상변수 Spearman 히트맵 |

### 핵심 발견
- **프로모터 메틸화 → 발현 억제** (음의 cis-correlation)
- **유전자체 메틸화 → 전사 활성** (양의 cis-correlation)
- Cis 방식으로 검정 수 대폭 감소 → 통계적 파워 향상

**다음**: 04번 노트북 (MOFA 잠재 요인 분석 + GSEA)